In [4]:
# modulos
import sympy as sp
import numpy as np

# simbolos generales
s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy=sp.symbols("s,R1,R2,R3,R4,R5,R6,C1,C2,C3,C4,Vi,Vo,Vx,V,k,Vy")

g1=1/R1
g2=1/R2
g3=1/R3
g4=1/R4
X2=1/(s*C2)

# print rapido de ecuaciones
def pp(eqs):
    print("\n")
    [sp.pprint(eq,wrap_line=False) for eq in eqs]
    print("\n")

# print rapido de separadores
def pr(txt=None):
    # si no hay texto, imprimir linea de fin
    if txt==None:
        print(f"{'*'*100}\n")
    
    # sino, imprimir un cartel de inicio
    else:
        tittle=f"|{txt}|".center(2*len(txt))
        
        print("-"*len(tittle))
        print(tittle)
        print("-"*len(tittle))


# Sallen-Key
!["sallenKey"](./img/sallenKeyBP.jpg)

In [9]:
# ecuacion de circuito

# print("-"*32)
# print("|\tEcuaciones de nodos.\t|")
# print("-"*32)
pr("Ecuaciones de nodos.")

nodo0=sp.Eq(Vx*(g1+g2+s*C1+s*C2)-Vo*g2-(Vo/k)*s*C2-g1*Vi,0)
nodo1=sp.Eq((Vo/k)*(s*C2+g3)-s*C2*Vx,0)
pp([nodo0,nodo1])

sp.print_latex(nodo0)
sp.print_latex(nodo1)

pr()

# -----------------------------------------------------------------------------------
# buscar funcion de transferencia
pr("Funcion de transferencia.")

# resolver para vo y vi, y dividir
sol0=sp.solve((nodo0,nodo1),(Vo,Vi))

T=sp.collect(sp.simplify(sol0[Vo]/sol0[Vi]),s)
pp([sp.Eq(Vo/Vi,T)])

pr()

# -----------------------------------------------------------------------------------
# normalizar 
pr("Normalizacion.")

# recuperar num den
n,d=sp.fraction(T)

# normalizar coeficiente principal del denominador
n=sp.simplify(n/d.coeff(s,2))
d=sp.simplify(d/d.coeff(s,2))

T=sp.collect(n/d,s)
pp([sp.Eq(sp.symbols("T"),T)])

sp.print_latex(T)

pr()

# -----------------------------------------------------------------------------------
pr("Simplificaciones.")

# simbolos auxiliares
w1,w12,w2,w21,w31,w32=sp.symbols("w1,w12,w2,w21,w31,w32")

# simplificar ft
T0=T.subs({1/(R1*C1):w1,1/(R2*C2):w2}).subs({1/(R1*C2):w12,
                                             1/(R2*C1):w21,
                                             1/(R3*C1):w31,
                                             1/(R3*C2):w32})
pp([sp.Eq(T,T0)])

pr()

----------------------------------------
         |Ecuaciones de nodos.|         
----------------------------------------


  C₂⋅Vo⋅s      ⎛              1    1 ⎞   Vo   Vi    
- ─────── + Vx⋅⎜C₁⋅s + C₂⋅s + ── + ──⎟ - ── - ── = 0
     k         ⎝              R₂   R₁⎠   R₂   R₁    
              ⎛       1 ⎞    
           Vo⋅⎜C₂⋅s + ──⎟    
              ⎝       R₃⎠    
-C₂⋅Vx⋅s + ────────────── = 0
                 k           


- \frac{C_{2} Vo s}{k} + Vx \left(C_{1} s + C_{2} s + \frac{1}{R_{2}} + \frac{1}{R_{1}}\right) - \frac{Vo}{R_{2}} - \frac{Vi}{R_{1}} = 0
- C_{2} Vx s + \frac{Vo \left(C_{2} s + \frac{1}{R_{3}}\right)}{k} = 0
****************************************************************************************************

--------------------------------------------------
           |Funcion de transferencia.|            
--------------------------------------------------


Vo                                         C₂⋅R₂⋅R₃⋅k⋅s                                      
── = 

## Sensitividades

In [13]:
pr("Parametros de analisis.")
n0,d0=sp.fraction(T0)

wp2=sp.factor(d0.coeff(s,0))
bwp=sp.factor(d0.coeff(s,1))

pp([sp.Eq(sp.sympify("wp**2"),wp2),sp.Eq(sp.sympify("bwp"),bwp)])

pr()

# -----------------------------------------------------------------------------------------
print("\nSensitividad para wp:")

""" entender la operatoria para un parametro, por ej R1 """
# w1 == 1/(R1*C1) == (R1*C1)**-1
# w1ssR1=-1
# w32ssR1=0
# w2ssR1=0
# w31ssR1=0

# propiedad: (p1*p2)Sx == p1Sx + p2Sx
# w1w32ssR1=w1ssR1+w32ssR1
# w2w31ssR1=w2ssR1+w31ssR1

# propiedad: (p1+p2)Sx == (p1*p1Sx + p2*p2Sx) / (p1+p2)
# wp2ssR1=(w1*w32*w1w32ssR1 + w2*w31*w2w31ssR1)/(w1*w32 + w2*w31)

# propiedad: (p**n)Sx == n*(pSx)
# wpssR1=wp2ssR1/2

""" y generalizar la operatoria para cualquier parametro"""
# formato: pssx == [pssR1, pssR2,... ,pssRn, pssC1, pssC2,... , pssCn]
w1ssx=np.array([-1,0,0,-1,0])
w32ssx=np.array([0,0,-1,0,-1])
w2ssx=np.array([0,-1,0,0,-1])
w31ssx=np.array([0,0,-1,-1,0])

# propiedad: (p1*p2)Sx == p1Sx + p2Sx
w1w32ssx=w1ssx+w32ssx
w2w31ssx=w2ssx+w31ssx

# propiedad: (p1+p2)Sx == (p1*p1Sx + p2*p2Sx) / (p1+p2)
wp2ssx=(w1*w32*w1w32ssx + w2*w31*w2w31ssx)/(w1*w32 + w2*w31)

# propiedad: (p**n)Sx == n*(pSx)
wpssx=wp2ssx/2

# reemplazar por componentes
wpssx=[sp.simplify(ssx.subs({w1:1/(R1*C1),
                             w2:1/(R2*C2),
                             w12:1/(R1*C2),
                             w21:1/(R2*C1),
                             w31:1/(R3*C1),
                             w32:1/(R3*C2)})) for ssx in wpssx]

# generar un diccionario
keys=[R1,R2,R3,C1,C2]

# aux={}
# for kk in range(len(keys)):
#     aux.update({keys[kk]:wpssx[kk]})
wpStv={k:v for k,v in zip(keys,wpssx)}
pp([wpStv])

# comprobar los calculos con identidad para wp
# identidad: sum(wpssC) == sum(wpssR) == -1

# wpSumR=sp.simplify(wpStv[R1]+wpStv[R2]+wpStv[R3])
# wpSumC=sp.simplify(wpStv[C1]+wpStv[C2])
# pp([sp.Eq(sp.Sum(sp.symbols("wpSsR_i"), (sp.symbols("i"), 1, 3)),wpSumR)])
# pp([sp.Eq(sp.Sum(sp.symbols("wpSsC_i"), (sp.symbols("i"), 1, 2)),wpSumC)])

pr()

# -----------------------------------------------------------------------------------------
pr("Sensitividad para bwp.")

""" bwp == -k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂ == w₂₁⋅(1-k) + w₁ + w₃₁ + w₃₂ """
# sensitividades basicas
w1ssx=w1ssx
w21ssx=np.array([0,-1,0,-1,0])
w31ssx=w31ssx
w32ssx=w32ssx

# propiedad: (c*p)Sx == pSx
cte_w21ssx=w21ssx

# propiedad: (p1+p2+ ...+pn)Sx == (p1*p1Sx + p2*p2Sx + ...+ pn*pnSx) / (p1+p2+ ...+pn)
bwpssx=((1-k)*cte_w21ssx + w1*w1ssx + w31*w31ssx + w32*w32ssx)/(bwp)
pp([bwp])



# reemplazar por componentes
w1=1/(R1*C1)
w2=1/(R2*C2)
w12=1/(R1*C2)
w21=1/(R2*C1)
w31=1/(R3*C1)
w32=1/(R3*C2)

bwpssx=[bwpss.subs({bwp:sp.symbols("bw"),
                                w1:w1,
                                w2:w2,
                                w12:w12,
                                w21:w21,
                                w31:w31,
                                w32:w32}) for bwpss in bwpssx]

# generar diccionario
bwpStv={k:v for k,v in zip(keys,bwpssx)}

# comprobar los calculos con identidad para wp
# identidad: sum(bwpssC) == sum(bwpssR) == 0

bwpSumR=sp.simplify(bwpStv[R1]+bwpStv[R2]+bwpStv[R3])
bwpSumC=sp.simplify(bwpStv[C1]+bwpStv[C2])
pp([sp.Eq(sp.Sum(sp.symbols("bwpSsR_i"), (sp.symbols("i"), 1, 3)),bwpSumR)])
pp([sp.Eq(sp.Sum(sp.symbols("bwpSsC_i"), (sp.symbols("i"), 1, 2)),bwpSumC)])

pp([bwpStv])

pr()

----------------------------------------------
          |Parametros de analisis.|           
----------------------------------------------


  2                  
wp  = w₁⋅w₃₂ + w₂⋅w₃₁
bwp = -k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂


****************************************************************************************************


Sensitividad para wp:


⎧                            -R₂              -R₁              ⎫
⎨C₁: -1/2, C₂: -1/2, R₁: ───────────, R₂: ───────────, R₃: -1/2⎬
⎩                        2⋅R₁ + 2⋅R₂      2⋅R₁ + 2⋅R₂          ⎭


****************************************************************************************************

--------------------------------------------
          |Sensitividad para bwp.|          
--------------------------------------------


-k⋅w₂₁ + w₁ + w₂₁ + w₃₁ + w₃₂


1/(C1*R3)


  3                                                       
 ___                                                      
 ╲                                                

## Diseño

In [52]:
# requisotos
k0=5252.26 
a0=3184.42 
b0=66320656

k1=3126.49 
a1=1895.58 
b1=23500096

# funciones de transferencia
print("Transferencias.")

T0=T
T1=T.subs({R1:R4,R2:R5,R3:R6,C1:C3,C2:C4})

sp.pprint(sp.Eq(T0,k0*s/(s**2+a0*s+b0)),wrap_line=False)
sp.pprint(sp.Eq(T1,k1*s/(s**2+a1*s+b1)),wrap_line=False)
sp.print_latex(sp.Eq(T0,-k0*s/(s**2+a0*s+b0)))
print("-"*100)

# igualacion de coef
print("Igualaciones.")

n0,d0=sp.fraction(T0)

ig0=sp.Eq((d0/(R1*C1)).coeff(s,1),a0)
ig1=sp.Eq((d0/(R1*C1)).coeff(s,0),b0)
sp.pprint(ig0)
sp.pprint(ig1)
sp.print_latex(ig0)
sp.print_latex(ig1)

n1,d1=sp.fraction(T1)

ig2=sp.Eq((d1/(R4*C3)).coeff(s,1),a1)
ig3=sp.Eq((d1/(R4*C3)).coeff(s,0),b1)
sp.pprint(ig2)
sp.pprint(ig3)
print("-"*100)

# valuando componentes

print("Valuando")

c1=1
c2=1
R=sp.symbols("R")

ig0=ig0.subs({C1:c1,C2:c2,R1:R,R2:R,R3:R})
ig1=ig1.subs({C1:c1,C2:c2,R1:R,R2:R,R3:R})
sp.pprint(ig0)
sp.pprint(ig1)
sp.print_latex(ig0)
sp.print_latex(ig1)

c3=1
c4=1

ig2=ig2.subs({C3:c3,C4:c4,R4:R,R5:R,R6:R})
ig3=ig3.subs({C3:c3,C4:c4,R4:R,R5:R,R6:R})
sp.pprint(ig2)
sp.pprint(ig3)
print("-"*100)

# calculo de componentes
print("Resultados.")

# ignorar parte negativa de las raices
(r0,k0)=sp.solve((ig0,ig1),(R,k))[1]
(r1,k1)=sp.solve((ig2,ig3),(R,k))[1]


# componentes
escala0=10E6
escala1=10E6

print("Filtro0")
sp.pprint(sp.Eq(C1,c1/escala0))
sp.pprint(sp.Eq(C2,c2/escala0))
sp.pprint(sp.Eq(R,r0*escala0))
sp.pprint(sp.Eq(k,k0))
sp.print_latex(sp.Eq(C1,c1/escala0))
sp.print_latex(sp.Eq(C2,c2/escala0))
sp.print_latex(sp.Eq(R,r0*escala0))
sp.print_latex(sp.Eq(k,k0))

print("Filtro1")
sp.pprint(sp.Eq(C3,c3/escala1))
sp.pprint(sp.Eq(C4,c4/escala1))
sp.pprint(sp.Eq(R,r1*escala1))
sp.pprint(sp.Eq(k,k1))
print("-"*100)

print("Ajuste de ganancia.")
# atenuacion en dB
att=28.9
# en veces
print(10**(att/20))
print("-"*100)

Transferencias.
                                       k⋅s                                                   5252.26⋅s        
────────────────────────────────────────────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1       k       1       1  ⎞        1             1     ⎞    2                       
C₁⋅R₁⋅⎜s  + s⋅⎜───── + ───── - ───── + ───── + ─────⎟ + ─────────── + ───────────⎟   s  + 3184.42⋅s + 66320656
      ⎝       ⎝C₂⋅R₃   C₁⋅R₃   C₁⋅R₂   C₁⋅R₂   C₁⋅R₁⎠   C₁⋅C₂⋅R₂⋅R₃   C₁⋅C₂⋅R₁⋅R₃⎠                            
                                       k⋅s                                                   3126.49⋅s        
────────────────────────────────────────────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1       k       1       1  ⎞        1             1     ⎞    2                       
C₃⋅R₄⋅⎜s  + s⋅⎜───── + ───── - ───── + ───── + ─────⎟ + ─────────── + ───────────⎟   s  + 1895.5

# Negative feedback
!["negFeed"](./img/negFeedBP.jpg)

In [5]:
# ecuacion de circuito
print("-"*32)
print("|\tEcuaciones de nodos.\t|")
print("-"*32)

# considerando el nodo X (R1,C1,C2)
nodo0=sp.Eq(g1*Vi,Vx*(g1+s*C1+s*C2)-Vo*s*C2)
# teniendo en cuenta que la corriente por C1 debe ser igual a la corriente por R2
nodo1=sp.Eq(Vo*g2,-Vx*s*C1)

sp.pprint(nodo0)
sp.pprint(nodo1)
sp.print_latex(nodo0)
sp.print_latex(nodo1)
print("-"*100)

print("-"*37)
print("|\tFuncion de transferencia.\t|")
print("-"*37)

eq0=nodo0.subs({Vx:sp.solve(nodo1,Vx)[0]})
sp.pprint(eq0)

# resolver para vo
eq1=sp.solve(eq0,Vo)[0]

# dividir por vi
T=eq1/Vi
T=sp.simplify(sp.Eq(Vo/Vi,T))
sp.pprint(T,wrap_line=False)

# recuperar nominador y denom
n0,d0=sp.fraction(T.rhs)

# normalizar
n0=sp.collect(n0/d0.coeff(s,2),s)
d0=sp.collect(sp.simplify(d0/d0.coeff(s,2)),s)
sp.pprint(n0,wrap_line=False)
sp.pprint(d0,wrap_line=False)

T=n0/d0
sp.pprint(T,wrap_line=False)
sp.print_latex(T)


--------------------------------
|	Ecuaciones de nodos.	|
--------------------------------
Vi                 ⎛              1 ⎞
── = -C₂⋅Vo⋅s + Vx⋅⎜C₁⋅s + C₂⋅s + ──⎟
R₁                 ⎝              R₁⎠
Vo           
── = -C₁⋅Vx⋅s
R₂           
\frac{Vi}{R_{1}} = - C_{2} Vo s + Vx \left(C_{1} s + C_{2} s + \frac{1}{R_{1}}\right)
\frac{Vo}{R_{2}} = - C_{1} Vx s
----------------------------------------------------------------------------------------------------
-------------------------------------
|	Funcion de transferencia.	|
-------------------------------------
                   ⎛              1 ⎞
                Vo⋅⎜C₁⋅s + C₂⋅s + ──⎟
Vi                 ⎝              R₁⎠
── = -C₂⋅Vo⋅s - ─────────────────────
R₁                     C₁⋅R₂⋅s       
Vo                 -C₁⋅R₂⋅s                
── = ──────────────────────────────────────
Vi                2                        
     C₁⋅C₂⋅R₁⋅R₂⋅s  + C₁⋅R₁⋅s + C₂⋅R₁⋅s + 1
 -s  
─────
C₂⋅R₁
 2     ⎛  1       1  ⎞        1     
s  + s⋅

In [9]:
# requisotos
k0=5252.26 
a0=3184.42 
b0=66320656

k1=3126.49 
a1=1895.58 
b1=23500096

# funciones de transferencia
print("Transferencias.")

T0=T
T1=T.subs({R1:R3,R2:R4,C1:C3,C2:C4})
sp.pprint(sp.Eq(T0,-k0*s/(s**2+a0*s+b0)))
sp.pprint(sp.Eq(T1,-k1*s/(s**2+a1*s+b1)))
sp.print_latex(sp.Eq(T1,-k1*s/(s**2+a1*s+b1)))
print("-"*100)

# igualacion de coef
print("Igualaciones.")

ig0=sp.Eq(1/(R2*C2)+1/(R2*C1),a0)
ig1=sp.Eq(1/(R2*C2)*1/(R1*C1),b0)
sp.pprint(ig0)
sp.pprint(ig1)
sp.print_latex(ig0)
sp.print_latex(ig1)

ig2=sp.Eq(1/(R4*C4)+1/(R4*C3),a1)
ig3=sp.Eq(1/(R4*C4)*1/(R3*C3),b1)
sp.pprint(ig2)
sp.pprint(ig3)
print("-"*100)

# valuando componentes

print("Valuando")

c1=1
c2=1

ig0=ig0.subs({C1:c1,C2:c2})
ig1=ig1.subs({C1:c1,C2:c2})
sp.pprint(ig0)
sp.pprint(ig1)
sp.print_latex(ig0)
sp.print_latex(ig1)

c3=1
c4=1

ig2=ig2.subs({C3:c3,C4:c4})
ig3=ig3.subs({C3:c3,C4:c4})
sp.pprint(ig2)
sp.pprint(ig3)
print("-"*100)

# calculo de componentes
print("Resultados.")

[(r1,r2)]=sp.solve((ig0,ig1),(R1,R2))
[(r3,r4)]=sp.solve((ig2,ig3),(R3,R4))

# componentes
escala0=10E6
escala1=10E6

sp.pprint(sp.Eq(C1,c1/escala0))
sp.pprint(sp.Eq(C2,c2/escala0))
sp.pprint(sp.Eq(R1,r1*escala0))
sp.pprint(sp.Eq(R2,r2*escala0))

sp.pprint(sp.Eq(C3,c3/escala1))
sp.pprint(sp.Eq(C4,c4/escala1))
sp.pprint(sp.Eq(R3,r3*escala1))
sp.pprint(sp.Eq(R4,r4*escala1))
print("-"*100)

print("Ajuste de ganancia.")
# atenuacion en dB
att=35
# en veces
print(10**(att/20))
print("-"*100)

Transferencias.
                    -s                                -5252.26⋅s        
──────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1  ⎞        1     ⎞    2                       
C₂⋅R₁⋅⎜s  + s⋅⎜───── + ─────⎟ + ───────────⎟   s  + 3184.42⋅s + 66320656
      ⎝       ⎝C₂⋅R₂   C₁⋅R₂⎠   C₁⋅C₂⋅R₁⋅R₂⎠                            
                    -s                                -3126.49⋅s        
──────────────────────────────────────────── = ─────────────────────────
      ⎛ 2     ⎛  1       1  ⎞        1     ⎞    2                       
C₄⋅R₃⋅⎜s  + s⋅⎜───── + ─────⎟ + ───────────⎟   s  + 1895.58⋅s + 23500096
      ⎝       ⎝C₄⋅R₄   C₃⋅R₄⎠   C₃⋅C₄⋅R₃⋅R₄⎠                            
- \frac{s}{C_{4} R_{3} \left(s^{2} + s \left(\frac{1}{C_{4} R_{4}} + \frac{1}{C_{3} R_{4}}\right) + \frac{1}{C_{3} C_{4} R_{3} R_{4}}\right)} = - \frac{3126.49 s}{s^{2} + 1895.58 s + 23500096}
-------------------------------------------------------------

# Operaciones auxiliares

In [38]:
#  simbolos extras
RA,RB=sp.symbols("RA,RB")
RB=RA/(k-1)
ga=1/RA
gb=1/RB

# ecuacion de circuito
print("-"*32)
print("|\tEcuaciones de nodos.\t|")
print("-"*32)

nodoX=sp.Eq(Vx*(s*C1+g1+1/(R2+X2))-Vi*s*C1-Vo*1/(R2+X2),0)
nodoY=sp.Eq(Vy*(ga+gb)-Vo*ga,0)
# sp.pprint(nodoX)
# sp.pprint(nodoY)
pp([nodoX,nodoY])
print("-"*100)

# condicion de operacional ideal: Vy==Vx
print("condicion de operacional ideal: Vy==Vx")
nodoY=nodoY.subs(Vy,Vx)
pp([nodoX,nodoY])

# obtener soluciones
sol0=sp.solve((nodoX,nodoY),(Vo,Vi))
# recuperar num-den
n0,d0=sp.fraction(sp.collect(sp.simplify(sol0[Vo]/sol0[Vi]),s))
# normalizar
n0=sp.collect(sp.expand(n0/d0.coeff(s,2)),s)
d0=sp.collect(sp.expand(d0/d0.coeff(s,2)),s)

# funcion de transferencia final
T0=sp.factor(n0)/d0

pp([T0])

--------------------------------
|	Ecuaciones de nodos.	|
--------------------------------
               Vo         ⎛           1       1 ⎞    
-C₁⋅Vi⋅s - ───────── + Vx⋅⎜C₁⋅s + ───────── + ──⎟ = 0
                 1        ⎜             1     R₁⎟    
           R₂ + ────      ⎜       R₂ + ────     ⎟    
                C₂⋅s      ⎝            C₂⋅s     ⎠    
   ⎛k - 1   1 ⎞   Vo    
Vy⋅⎜───── + ──⎟ - ── = 0
   ⎝  RA    RA⎠   RA    
----------------------------------------------------------------------------------------------------
condicion de operacional ideal: Vy==Vx
               Vo         ⎛           1       1 ⎞    
-C₁⋅Vi⋅s - ───────── + Vx⋅⎜C₁⋅s + ───────── + ──⎟ = 0
                 1        ⎜             1     R₁⎟    
           R₂ + ────      ⎜       R₂ + ────     ⎟    
                C₂⋅s      ⎝            C₂⋅s     ⎠    
   ⎛k - 1   1 ⎞   Vo    
Vx⋅⎜───── + ──⎟ - ── = 0
   ⎝  RA    RA⎠   RA    
                     k⋅s⋅(C₂⋅R₂⋅s + 1)                      
──────────────────